This code makes an automatic implementation of the probability of defauls of an open company in US based on Merton's Model. The methodology was based on the reference given below.

The Merton's Model values the equity of a company in a time T as a option that allows you to earn 
                máx(Assets- Liabilities, recovery_rate*(assets))

Let's assume that the recovery rate is 0.

Using option pricing, we reach into a two equations system:

E0 = A0*Normal(d1) - L0*exp(-r*T)Normal(d2)
sigmaE0 = (A0/E0) * Normal(d1) * sigmaA0

Where:
E0 = equity at time zero
A0 = asset at time zero
r = risk free rate
T = time of the maturity of the liability
sigmaE0 = volatility of the equity
sigmaA0 = volatility of the asset

d1 = (1/sigmaA0*sqrt(T))* [log(A0/E0) + (r + (sigmaA0^2)/2)*T]
d2 = d1 - sigmaA0 * T



We have the E0 (from stock market) and we have sigmaE0. We don't have A0 and sigmaA0. To find these values we are going to use the two equations.


Reference: 

1 - https://www.mathworks.com/help/risk/default-probability-using-the-merton-model-for-structural-credit-risk.html?requestedDomain=www.mathworks.com

2 - Risk Book



In [321]:
import good_morning as gm                 # Morningstar package to get financial info
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import norm
import numpy as np

In [322]:
# This function returns the historical total equity. 

def get_Historical_Total_Equity1(Stock):

    kr = gm.FinancialsDownloader()
    kr_frames = kr.download(Stock)
    E0 = kr_frames['balance_sheet'][(kr_frames['balance_sheet']['title'] == 'Total Stockholders\' equity')]

    return E0

def get_Historical_Total_Equity2(Stock):

    kr = gm.FinancialsDownloader()
    kr_frames = kr.download(Stock)
    E0 = kr_frames['balance_sheet'][(kr_frames['balance_sheet']['title'] == 'Total stockholders\' equity')]

    return E0

In [323]:
# This function downloads the historical stock closing price and retunrs the volatility. THis will be our sigmaE0

def get_Volatility(Stock, delta_time):
    url = "https://app.quotemedia.com/quotetools/getHistoryDownload.csv?&webmasterId=501&startDay=02&startMonth=02&startYear=2002&endDay=22&endMonth=09&endYear=2017&isRanged=false&symbol={}".format(Stock)
    
    data = pd.read_csv(url, parse_dates = True, index_col = 0)
    data = data.loc[:,('close')]
    data = data.sort_index(ascending = False)
    
    sigmaE0 = np.std(data.tail(delta_time))
    
    return sigmaE0

In [324]:
# The maturity of the liability is tricky. This function makes a weighted Time of maturity. From the financials
# we have current libalities and non_current liabilities. The first one is less than one year and the second one
# is over one year maturity. 

def get_Maturity(Stock):
    kr = gm.FinancialsDownloader()
    kr_frames = kr.download(Stock)
    current_liability = kr_frames['balance_sheet'][(kr_frames['balance_sheet']['title'] == 'Current liabilities')].iloc[0,-1]
    non_current_liability = kr_frames['balance_sheet'][(kr_frames['balance_sheet']['title'] == 'Non-current liabilities')].iloc[0,-1]
    time = (current_liability*0.6 + non_current_liability*2)/(current_liability + non_current_liability)
    
    return time

In [325]:
# Now let's get the initial liability

def get_Debt_Face_Value(Stock):
    
    kr = gm.FinancialsDownloader()
    kr_frames = kr.download(Stock)
    L0 = kr_frames['balance_sheet'][(kr_frames['balance_sheet']['title'] == 'Liabilities')].iloc[0,-1]

    return L0


In [326]:
# Finally, the model. 

def merton_solve(param, E0, sigmaE0, liability, Maturity):
    A0 = param[0]
    sigmaA0 = param[1]
    
    T = Maturity
    B0 = liability
    
    d1 = (np.log(A0/B0) + (r+(sigmaA0**2)/2)*T)/(sigmaA0*(T**(1/2)))
    d2 = d1 - sigmaA0*(T**(1/2))
    
    F = A0*norm.cdf(d1) - B0*np.exp(-r*T)*norm.cdf(d2) - E0         # This is the option pricin equation. F must be 0
    G = norm.cdf(d1)*sigmaA0*A0 - sigmaE0*E0                         # This is the volatility 
    
    return F**2 + G**2      # We want to minimize this function

In [327]:
def get_prob_default(A0,sigmaA0, E0, sigmaE0, liability, Maturity):
    T = Maturity
    B0 = liability
    
    d1 = (np.log(A0/B0) + (r+(sigmaA0**2)/2)*T)/(sigmaA0*(T**(1/2)))
    d2 = d1 - sigmaA0*(T**(1/2))
    
    probability = norm.cdf(d2)
    
    return probability

In [328]:
stock = "AAPL"

In [329]:
try: 
    E0 = get_Historical_Total_Equity1(stock).iloc[0,-1]
except IndexError:
    pass

try: 
    E0 = get_Historical_Total_Equity2(stock).iloc[0,-1]
except IndexError:
    pass

E0

128249000000.0

In [319]:
sigmaE0 = get_Volatility(stock, 100)
sigmaE0

0.4975204417910888

In [320]:
T = get_Maturity(stock)
T

IndexError: single positional indexer is out-of-bounds

In [315]:
Liability = get_Debt_Face_Value(stock)
Liability

2372390000.0

In [316]:
param = [Liability + E0, sigmaE0] # Inicial guess


In [306]:
r = 0.05 # risk free rate

In [307]:
res = minimize(merton_solve, param,args = ( E0, sigmaE0, Liability, T))

In [308]:
get_prob_default(res['x'][0],res['x'][1],E0, sigmaE0, Liability, T)

0.045227652389018984